<div>
<center><h1>Twitter Response Bot</h1><br><img src="http://cdn.ultraswank.net/uploads/Monty_Python_and_the_Holy_Grail_2-1000x601.jpg" width=600></div>

In [65]:
import os, re, time
from markovbot import MarkovBot
from nltk.stem import PorterStemmer
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV

<center><h1>Load the Model Classifier</h1>

In [84]:
clf = joblib.load('model/hate-speech-classifier.pkl')
vec = joblib.load('model/hate-speech-vector.pkl')

In [88]:
class PorterTokenizer(object):
    """Custom PorterTokenizer for TfidfVectorizer"""

    def __init__(self):
        self.stemmer = PorterStemmer()

    def __call__(self, doc):
        translate_table = dict((ord(char), None) for char in string.punctuation)
        return [self.stemmer.stem(t) for t in word_tokenize(doc.translate(translate_table))]

class Vectorizer(object):
    """Vecotizer wrapper for sklearn TfidfVectorizer.

    Allows passing of custom tokenizer

    TODO: add more custom tokenizers"""

    def __init__(self,
                 tokenizer=None,
                 encoding='utf-8',
                 stop_words='english',
                 min_df=1,
                 ngram_range=None):
        self.tokenizers = {'porter': PorterTokenizer()}
        self.vectorizer = TfidfVectorizer(tokenizer=self.tokenizers[tokenizer],
                                          encoding=encoding,
                                          stop_words=stop_words,
                                          min_df=min_df,
                                          ngram_range=ngram_range)

    def fit(self, X):
        self.vectorizer.fit(X)
        return self

    def fit_transform(self, X):
        return self.vectorizer.fit_transform(X)

    def transform(self, X):
        return self.vectorizer.transform(x)
    
def test_model(base_model, param_grid):
    grid_clf = GridSearchCV(base_model, param_grid, cv=5)
    grid_clf.fit(train_features, y_train)
    preds = grid_clf.predict(test_features)
    print(classification_report(y_test, preds))
    return grid_clf

def top_words(clf, label, top):
    data = []
    for i in clf.best_estimator_.coef_[label, :].argsort()[::-1][:top]:
        top_words = (i, clf.best_estimator_.coef_[0, i], vec.vectorizer.get_feature_names()[i])
        data.append("{}".format(top_words[2]))
    return data

In [89]:
print(clf)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [1, 10, 100, 1000]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)


In [92]:
hate_words = top_words(clf, 0, 15)

<center><h1>Bot Code</h1>

In [36]:
bot = MarkovBot()
# read input of text replies and build random conversational sentences
bot.read('replies.txt')
text = bot.generate_text(15, seedword=[''])
print('bot says: '+text)

bot says: We do routines and chorus scenes With footwork impeccable. We dine well here in Camelot.


In [ ]:
# Twitter API stuff
cons_key = 'fai4h6SN6gFM9nhQGu2A72mHY'
cons_secret = 'GQLWRbIMeVcT6BT1uLP79oA2JmS7WUyZUgN7KoHZGPG4v7KOPi'
access_token = '848222105535602688-doZSwW16KKC1PQd4YKenIuYUTe6n4nh'
access_token_secret = 'MNI1HQDUIIJ01AKkjyyopSdz7QNlhVG3Ryox1Fpv3TPi2'

In [ ]:
# Log in to Twitter
bot.twitter_login(cons_key, cons_secret, access_token, access_token_secret)

In [ ]:
prefix = None
suffix = ''
maxconvdepth = 2
bot.twitter_autoreply_start(targetstring = '', keywords=hate_words, prefix=prefix, suffix=suffix, maxconvdepth=maxconvdepth)
bot.twitter_tweeting_start(days=0, hours=19, minutes=30, keywords=None, prefix=None, suffix='#BleepBloop')
secsinweek = 60
time.sleep(secsinweek)

In [ ]:
tweetbot.twitter_autoreply_stop()
tweetbot.twitter_tweeting_stop()